In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys

sys.path.append("..")
import pandas as pd 
import numpy as np 
from src.processing import hrrr_data
from src.processing import nysm_data
from src.processing import get_error
from src.processing import normalize
from src.processing import get_flag
# from src.processing import create_data_for_vision
import pandas as pd

In [6]:
nysm_cats_path = "/home/aevans/nwp_bias/src/landtype/data/nysm.csv"
nysm_cats_df = pd.read_csv(nysm_cats_path)

In [7]:
nysm_cats_df['climate_division_name'].unique()

array(['Western Plateau', 'Eastern Plateau', 'Great Lakes',
       'Hudson Valley', 'Coastal', 'Central Lakes', 'Mohawk Valley',
       'Champlain Valley', 'Northern Plateau', 'St. Lawrence Valley'],
      dtype=object)

In [10]:
def columns_drop(df):
    df = df.drop(
        columns=[
            "level_0",
            "index_x",
            "index_y",
            "lead time",
            "lsm",
            "station_y",
            'lat',
            'lon'
        ]
    )
    df = df.rename(columns={'station_x':'station'})
    return df

In [13]:
def create_data_for_model(clim_div, today_date):
    """
    This function creates and processes data for a vision transformer machine learning model.

    Returns:
        df_train (pandas DataFrame): A DataFrame for training the machine learning model.
        df_test (pandas DataFrame): A DataFrame for testing the machine learning model.
        features (list): A list of feature names.
    """
    # load nysm data
    nysm_df = nysm_data.load_nysm_data()
    nysm_df.reset_index(inplace=True)
    nysm_df = nysm_df.rename(columns={"time_1H": "valid_time"})

    # load hrrr data
    hrrr_df = hrrr_data.read_hrrr_data()

    # Filter data by NY climate division
    nysm_cats_path = "/home/aevans/nwp_bias/src/landtype/data/nysm.csv"
    nysm_cats_df = pd.read_csv(nysm_cats_path)
    nysm_cats_df = nysm_cats_df[nysm_cats_df["climate_division_name"] == clim_div]
    stations = nysm_cats_df["stid"].tolist()
    nysm_df = nysm_df[nysm_df["station"].isin(stations)]
    hrrr_df = hrrr_df[hrrr_df["station"].isin(stations)]

    # need to create a master list for valid_times so that all the dataframes are the same shape
    master_time = hrrr_df["valid_time"].tolist()
    for station in stations:
        hrrr_dft = hrrr_df[hrrr_df["station"] == station]
        nysm_dft = nysm_df[nysm_df["station"] == station]
        times = hrrr_dft["valid_time"].tolist()
        times2 = nysm_dft["valid_time"].tolist()
        result = list(set(times) & set(master_time) & set(times2))
        master_time = result
    master_time_final = master_time

    # Filter NYSM data to match valid times from master-list
    nysm_df_filtered = nysm_df[nysm_df["valid_time"].isin(master_time_final)]
    hrrr_df_filtered = hrrr_df[hrrr_df["valid_time"].isin(master_time_final)]

    df_train_ls = []
    df_test_ls = []
    # merge dataframes so that each row is hrrr + nysm data for the same time step
    # do this for each station individually
    for station in stations:
        print(f"Compiling Data for {station}")
        nysm_df1 = nysm_df_filtered[nysm_df_filtered["station"] == station]
        hrrr_df1 = hrrr_df_filtered[hrrr_df_filtered["station"] == station]

        master_df = hrrr_df1.merge(nysm_df1, on="valid_time")
        master_df = columns_drop(master_df)

        # Calculate the error using NWP data.
        master_df = get_error.nwp_error("t2m", master_df)
        # encode for day_of_year
        master_df = normalize.encode(master_df, "day_of_year", 366)

        cols_to_carry = ["valid_time", "station", "latitude", "longitude"]
        # master_df.to_parquet(
        #     f"/home/aevans/transformer_ml/src/data/temp_df/{today_date}/{clim_div}/{clim_div}_{station}.parquet"
        # )

        new_df = master_df.drop(columns=cols_to_carry)

        # for c in new_df.columns:
        #     new_df[f'{c}_dt'] = new_df[c].diff()
        # new_df = new_df.fillna(0)
        new_df, features = normalize.normalize_df(new_df)
        new_df = new_df.fillna(0)

        # Split the data into training and testing sets.
        length = len(new_df)
        test_len = int(length * 0.8)
        df_train = new_df.iloc[:test_len].copy()
        df_test = new_df.iloc[test_len:].copy()
        print("Test Set Fraction", len(df_test) / len(new_df))

        # Reintegrate the specified columns back into the training and testing DataFrames.
        # for c in cols_to_carry:
        #     df_train[c] = master_df[c]
        #     df_test[c] = master_df[c]
        df_train_ls.append(df_train)
        df_test_ls.append(df_test)

        print("train_shape", df_train.shape)
        print("test_shape", df_test.shape)
        gc.collect()
        # print("train_start", df_train['valid_time'].iloc[0])
        # print("test_start", df_test['valid_time'].iloc[0])
    return df_train_ls, df_test_ls, features, stations

In [14]:
df_train, df_test, features, station = create_data_for_model('Central Lakes', 'test')

Compiling Data for BURD
Test Set Fraction 0.2000079242442252
train_shape (40382, 35)
test_shape (10096, 35)


NameError: name 'gc' is not defined

In [ ]:
nysm_df = nysm_data.load_nysm_data()
nysm_df.reset_index(inplace=True)
nysm_df = nysm_df.rename(columns={"time_1H": "valid_time"})
nysm_df.head()

In [ ]:
hrrr_df = hrrr_data.read_hrrr_data()
hrrr_df.head()

In [ ]:
# Set the path for tabular data.
nysm_cats_path = "/home/aevans/nwp_bias/src/landtype/data/nysm.csv"
nysm_cats_df = pd.read_csv(nysm_cats_path)
nysm_cats_df = nysm_cats_df[nysm_cats_df['climate_division_name']=='Western Plateau']
nysm_cats_df

In [ ]:
stations = nysm_cats_df["stid"].tolist()

In [ ]:
nysm_df = nysm_df[nysm_df['station'].isin(stations)]
hrrr_df = hrrr_df[hrrr_df['station'].isin(stations)]

In [ ]:
nysm_df.head()

In [ ]:
hrrr_df.head()

In [ ]:
# need to create a master list for valid_times so that all the dataframes are the same shape
master_time = hrrr_df['valid_time'].tolist()
for station in stations:
    hrrr_dft = hrrr_df[hrrr_df["station"] == station]
    nysm_dft = nysm_df[nysm_df["station"] == station]
    times = hrrr_dft['valid_time'].tolist()
    times2 = nysm_dft['valid_time'].tolist()
    result = list(set(times) & set(master_time) & set(times2))
    master_time = result

master_time_final = master_time

In [ ]:
sort_me1 = (sorted(master_time_final))

In [ ]:
sort_me1

In [ ]:
print(len(master_time_final))
sorted(master_time_final)

In [ ]:
print(len(master_time))
sorted(master_time)

In [ ]:
for station in stations:
    nysm_df1 = nysm_df[nysm_df['station']==station]
    hrrr_df1 = hrrr_df[hrrr_df['station']==station]

    master_df = hrrr_df1.merge(nysm_df1, on="valid_time")
    master_df = master_df.drop_duplicates(
        subset=["valid_time", "t2m"], keep="first"
    )

In [ ]:
master_df = columns_drop(master_df)

In [ ]:
master_df.head()

In [ ]:
# Calculate the error using NWP data.
master_df = get_error.nwp_error("t2m", master_df)

In [ ]:
master_df = normalize.encode(master_df, 'day_of_year', 366)

In [ ]:
master_df.head()

In [ ]:
master_df = get_flag.get_flag(master_df)

In [ ]:
master_df

In [ ]:
cols_to_carry = ['valid_time', 'station', 'latitude', 'longitude', 'flag']

In [ ]:
new_df = master_df.drop(columns=cols_to_carry)

In [ ]:
new_df, features = normalize.normalize_df(new_df)

In [ ]:
new_df.head()

In [ ]:
features

In [ ]:
# Split the data into training and testing sets.
length = len(new_df)
test_len = int(length * 0.8)
df_train = new_df.iloc[:test_len].copy()
df_test = new_df.iloc[test_len:].copy()
print("Test Set Fraction", len(df_test) / len(new_df))

In [ ]:
# Reintegrate the specified columns back into the training and testing DataFrames.
for c in cols_to_carry:
    df_train[c] = master_df[c]
    df_test[c] = master_df[c]

In [ ]:
df_train.head()

In [ ]:
df_test.head()